https://uis.fabric-testbed.net/ui/

https://uis.fabric-testbed.net/ui/#/people/people_uuid_get

ssh -i ~/work/fabric_config/pruth_fabric_rsa  -L 2001:localhost:9200  -F ~/work/fabric_config/ssh_config pruth@mfelk-p.fabric-testbed.net

In [ ]:
from reports_api import ReportsApi
from datetime import datetime, timezone
import json
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta

api =  ReportsApi(base_url="https://reports.fabric-testbed.net:8443/reports", token_file="/Users/kthare10/work/id_token_prod.json")
#api =  ReportsApi(base_url="http://localhost:8080/reports", token_file="/Users/kthare10/work/id_token_prod.json")

query_start=None

query_end = datetime.now(timezone.utc)
query_end = query_end.isoformat(timespec='milliseconds')

## Query Projects

In [ ]:
print("Query All projects")
projects = api.query_projects(fetch_all=False)
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

In [ ]:
print("Query All projects in a time range")
projects = api.query_projects(start_time=query_start, end_time=query_end, fetch_all=False)
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

In [ ]:
print("Query All projects using SmartNIC in a time range")
component_type="SmartNIC"
projects = api.query_projects(start_time=query_start, end_time=query_end, component_type=component_type, fetch_all=False)
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

In [ ]:

print("Query All projects using SmartNIC and ConnectX_5 in a time range")
component_type="SmartNIC"
component_model="ConnectX-5"
projects = api.query_projects(start_time=query_start, end_time=query_end, component_type=component_type, component_model=component_model, fetch_all=False)
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

In [ ]:
print("Query project using SmartNIC and ConnectX_5 in a time range for a specific project")
projects = api.query_projects(start_time=query_start, end_time=query_end, fetch_all=False, project_id="990d8a8b-7e50-4d13-a3be-0f133ffa8653")
print(f"Total projects: {projects.get('total')}")
print(json.dumps(projects.get("data")[0], indent=4))

## Query Users

In [ ]:
print("Query All users")
users = api.query_users(fetch_all=False)
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

In [ ]:
print("Query All users in a time range")
users = api.query_users(start_time=query_start, end_time=query_end, fetch_all=False)
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

In [ ]:
print("Query All users using SmartNIC in a time range")
component_type="SmartNIC"
users = api.query_users(start_time=query_start, end_time=query_end, component_type=component_type, fetch_all=False)
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

In [ ]:

print("Query All users using SmartNIC and ConnectX_5 in a time range")
component_type="SmartNIC"
component_model="ConnectX-5"
users = api.query_users(start_time=query_start, end_time=query_end, component_type=component_type, component_model=component_model, fetch_all=False)
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

In [ ]:
print("Query All users using SmartNIC and ConnectX_5 in a time range for a specific project")
users = api.query_users(start_time=query_start, end_time=query_end, fetch_all=False, project_id="990d8a8b-7e50-4d13-a3be-0f133ffa8653")
print(f"Total users: {users.get('total')}")
print(json.dumps(users.get("data")[0], indent=4))

## Query Slices

In [ ]:
print("Query All slices")
slices = api.query_slices(fetch_all=False)
print(f"Total slices: {slices.get('total')}")
print(json.dumps(slices.get("data")[0], indent=4))

In [ ]:
print("Query All slices in a time range")
slices = api.query_slices(start_time=query_start, end_time=query_end, fetch_all=False)
print(f"Total slices: {slices.get('total')}")
print(json.dumps(slices.get("data")[0], indent=4))

In [ ]:
print("Query All slices using SmartNIC in a time range")
component_type="SmartNIC"
slices = api.query_slices(start_time=query_start, end_time=query_end, component_type=component_type, fetch_all=False)
print(f"Total slices: {slices.get('total')}")
print(json.dumps(slices.get("data")[0], indent=4))

In [ ]:

print("Query All slices using SmartNIC and ConnectX_5 in a time range")
component_type="SmartNIC"
component_model="ConnectX-5"
slices = api.query_slices(start_time=query_start, end_time=query_end, component_type=component_type, component_model=component_model, fetch_all=False)
print(f"Total slices: {slices.get('total')}")
print(json.dumps(slices.get("data")[0], indent=4))

In [ ]:
print("Query All slices using SmartNIC and ConnectX_5 in a time range for a specific slice")
slices = api.query_slices(start_time=query_start, end_time=query_end, fetch_all=False, slice_id="f4bfccc3-1360-4148-90c0-b3f21b9240c2")
print(f"Total Slivers: {slices.get('total')}")
print(json.dumps(slices.get("data"), indent=4))

## Query Slivers

In [ ]:
print("Query All slivers")
slivers = api.query_slivers(fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:
print("Query VM slivers in a time range")
slivers = api.query_slivers(start_time=query_start, end_time=query_end, sliver_type=["VM"], fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:
print("Query VM slivers in a time range excluding CERN and CIEN")
slivers = api.query_slivers(start_time=query_start, end_time=query_end, sliver_type=["VM"], 
                            exclude_site=["CERN", "CIEN"], fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:
print("Query All slivers using SmartNIC in a time range")
component_type="SmartNIC"
slivers = api.query_slivers(start_time=query_start, end_time=query_end, component_type=component_type, fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:

print("Query All slivers using SmartNIC and ConnectX_5 in a time range")
component_type="SmartNIC"
component_model="ConnectX-5"
slivers = api.query_slivers(start_time=query_start, end_time=query_end, component_type=component_type, component_model=component_model, fetch_all=False)
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data")[0], indent=4))

In [ ]:
print("Query All slivers using SmartNIC and ConnectX_5 in a time range for a specific slice")
slivers = api.query_slivers(start_time=query_start, end_time=query_end, fetch_all=False, slice_id="7975644c-516d-4492-9420-a62010efac2d")
print(f"Total Slivers: {slivers.get('total')}")
print(json.dumps(slivers.get("data"), indent=4))

# Plots

## Slice per month

In [ ]:
# Step 2: Fetch all slices
response = api.query_slices(fetch_all=True)
slices = response["data"]

# Step 3: Create DataFrame and process dates
df = pd.DataFrame(slices)
df['lease_start'] = pd.to_datetime(df['lease_start'])

# Step 4: Drop rows with null lease_start
df = df.dropna(subset=['lease_start'])

# Step 5: Group by year and month
df['year_month'] = df['lease_start'].dt.to_period('M')
slices_per_month = df.groupby('year_month').size()

# Step 6: Plot
plt.figure(figsize=(10, 6))
slices_per_month.plot(kind='bar')
plt.title('Slices per Month')
plt.xlabel('Month')
plt.ylabel('Number of Slices')
plt.xticks(rotation=45)
plt.tight_layout()
plt.grid(axis='y')
plt.show()

## Active Slices per Month

In [ ]:
# Create DataFrame
df = pd.DataFrame(slices)

# Robust datetime parsing using 'mixed' format
df['lease_start'] = pd.to_datetime(df['lease_start'], utc=True, format='mixed')
df['lease_end'] = pd.to_datetime(df['lease_end'], utc=True, format='mixed')

# Generate monthly range
start_month = df['lease_start'].min().to_period('M').to_timestamp()
end_month = df['lease_end'].max().to_period('M').to_timestamp()
month_range = pd.date_range(start=start_month, end=end_month, freq='MS')

# Count active slices per month
active_counts = []
for month_start in month_range:
    month_start = month_start.tz_localize("UTC")
    month_end = (month_start + relativedelta(months=1)) - pd.Timedelta(seconds=1)
    active = df[(df['lease_start'] <= month_end) & (df['lease_end'] >= month_start)]
    active_counts.append(len(active))

# Plot
plt.figure(figsize=(12, 6))
plt.bar(month_range.strftime('%Y-%m'), active_counts)
plt.xticks(rotation=45)
plt.title("Active Slices Per Month")
plt.xlabel("Month")
plt.ylabel("Active Slice Count")
plt.tight_layout()
plt.grid(axis='y')
plt.show()


## Active Slices with SmartNICs

In [ ]:
# Step 1: Fetch all slices that used SmartNICs
response = api.query_slices(component_type="SmartNIC", fetch_all=True)
df = pd.DataFrame(response["data"])

# Step 2: Convert lease_start and lease_end to datetime (with UTC awareness)
# Robust datetime parsing using 'mixed' format
df['lease_start'] = pd.to_datetime(df['lease_start'], utc=True, format='mixed')
df['lease_end'] = pd.to_datetime(df['lease_end'], utc=True, format='mixed')

# Step 3: Define the month range for plotting
start_month = df['lease_start'].min().to_period('M').to_timestamp()
end_month = df['lease_end'].max().to_period('M').to_timestamp()
month_range = pd.date_range(start=start_month, end=end_month, freq='MS')

# Step 4: Count SmartNIC slices active in each month
smartnic_counts = []

for month_start in month_range:
    month_start = month_start.tz_localize("UTC")
    month_end = (month_start + relativedelta(months=1)) - pd.Timedelta(seconds=1)

    active_slices = df[
        (df['lease_start'] <= month_end) &
        (df['lease_end'] >= month_start)
    ]
    smartnic_counts.append(len(active_slices))

# Step 5: Plot the results
plt.figure(figsize=(12, 6))
plt.bar(month_range.strftime('%Y-%m'), smartnic_counts)
plt.xticks(rotation=45)
plt.title("SmartNIC Slice Usage Per Month")
plt.xlabel("Month")
plt.ylabel("Number of Active SmartNIC Slices")
plt.tight_layout()
plt.grid(axis='y')
plt.show()


## Top 10 users for SmartNIC in a time range

In [ ]:
import pandas as pd
from collections import defaultdict
from dateutil import parser

query_end = datetime.now(timezone.utc)
query_end = query_end.isoformat(timespec='milliseconds')

# Load slivers with SmartNICs
slivers = api.query_slivers(end_time=query_end, component_type="SmartNIC", fetch_all=True)["data"]

In [ ]:
# {user_id: {"usage": float, "user_email": str, "project_id": str, "project_name": str}}
user_usage = {}

for s in slivers:
    user_id = s.get("user_id")
    user_email = s.get("user_email")
    project_id = s.get("project_id")
    project_name = s.get("project_name")
    lease_start = s.get("lease_start")
    lease_end = s.get("lease_end")
    smartnic_count = s.get("components", {}).get("total", 0)

    if not user_id or not lease_start or not lease_end:
        continue

    try:
        start = pd.to_datetime(lease_start, utc=True)
        end = pd.to_datetime(lease_end, utc=True)
        hours = max((end - start).total_seconds() / 3600.0, 0)
    except Exception:
        continue

    usage = smartnic_count * hours

    if user_id not in user_usage:
        user_usage[user_id] = {
            "smartnic_usage_hours": 0,
            "user_email": user_email,
            "project_id": project_id,
            "project_name": project_name
        }

    user_usage[user_id]["smartnic_usage_hours"] += usage

# Convert to DataFrame
usage_df = pd.DataFrame([
    {
        "user_id": uid,
        "user_email": info["user_email"],
        "project_id": info["project_id"],
        "project_name": info["project_name"],
        "smartnic_usage_hours": info["smartnic_usage_hours"]
    }
    for uid, info in user_usage.items()
])

# Sort and display top 10
top_users = usage_df.sort_values("smartnic_usage_hours", ascending=False).head(10)
print(top_users)


## User to Slice and viceversa

In [ ]:
from collections import defaultdict

slices = api.query_slivers(end_time=query_end, fetch_all=True)["data"]

# Dictionary mapping user_id → {user_email, set of slice names}
user_slices = defaultdict(lambda: {"user_email": None, "slices": set()})

for s in slices:
    user_id = s.get("user_id")
    user_email = s.get("user_email")
    slice_name = s.get("slice_name")
    slice_id = s.get("slice_id")

    if not user_id or not slice_name:
        continue

    user_slices[user_id]["user_email"] = user_email
    user_slices[user_id]["slices"].add(f"{slice_name} ({slice_id})" if slice_id else slice_name)

print(len(user_slices))
# Print mapping
for user_id, info in user_slices.items():
    print(f"\nUser: {user_id} | Email: {info['user_email']}")
    print("  Slices:")
    for s in sorted(info["slices"]):
        print(f"    - {s}")